In [1]:
import numpy as np
import pandas as pd
import cv2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from deepface import DeepFace
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
#Loading dataset for the model --> DeepFace
data = pd.read_csv('fer2013.csv')

In [3]:
#Conver pixel coordinates to 2d images
def process_image(image):
    image = np.fromstring(image, sep=' ')
    image = image.reshape(48, 48, 1).astype('uint8')
    return image

In [4]:
# Data elaboration: from 2d images to numpy arrays
data['pixels'] = data['pixels'].apply(lambda x: process_image(x))
X = np.array(data['pixels'].tolist()) #Insert into training set only predictive attributes
y = data['emotion'] #Removing labes from the dataset

#Encoding labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
label_classes = label_encoder.classes_

In [5]:
# Since fer2013 has the emotion labeled as int we must convert them into string to calucate metrics with DeepFace
emotion_map = {
    0: 'angry',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'surprise',
    6: 'neutral'
}

inverse_emotion_map = {v: k for k, v in emotion_map.items()}

In [6]:
y_pred = []

#Foreach image in the dataset we parse it to deepface and calculate the emotion of the face
for i in tqdm(range(len(X)), desc="Processing images"):
    img = X[i]
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    result = DeepFace.analyze(img_rgb, actions=['emotion'], enforce_detection=False)
    emotion = result[0]
    y_pred.append(emotion['dominant_emotion'])

#We eliminated the unknonw labels
valid_indices = [i for i in range(len(y_pred)) if y_pred[i] in inverse_emotion_map]
y_encoded_filtered = y_encoded[valid_indices]
y_pred_filtered = [y_pred[i] for i in valid_indices]

#Ecoding of preditions
y_pred_encoded = [inverse_emotion_map[emotion] for emotion in y_pred_filtered]

#Metric calculation
accuracy = accuracy_score(y_encoded_filtered, y_pred_encoded)
precision = precision_score(y_encoded_filtered, y_pred_encoded, average='weighted')
recall = recall_score(y_encoded_filtered, y_pred_encoded, average='weighted')
f1 = f1_score(y_encoded_filtered, y_pred_encoded, average='weighted')

print(f"ACCURACY: {accuracy}")
print(f"PRECISION: {precision}")
print(f"RECALL: {recall}")
print(f"F1/SCORE: {f1}")

Processing images: 100%|██████████| 35887/35887 [40:00<00:00, 14.95it/s]  


ACCURACY: 0.8270404324685819
PRECISION: 0.8299920694733915
RECALL: 0.8270404324685819
F1/SCORE: 0.827524719950451
